In [1]:
"""
This document is adapted from the implementation of https://arxiv.org/pdf/1802.08667.pdf
by Rahul Singh, available here: https://github.com/r4hu1-5in9h/rrr/

This implementation is coded in Python, and calculates the debiased average derivative of the machine learner.
It extends the original implementation to find the debiased average derivative for a neural network 
directly from the automatic derivatives of the network, instead of the partial differences approach. 
"""

base = "/Users/max/Dropbox (MIT)/ag_production_functions/code/riesz_representer/rr_python/" 

import os
import sys
sys.path.append(os.path.abspath(base))
import numpy as np

In [6]:
from importlib import reload
import gen_dataset
import pandas as pd

In [15]:
#######################
# clean and format data
#######################

# Implements get_data
gen_dataset = reload(gen_dataset)

gas_df = pd.read_stata(base + 'gasoline_final_tf1.dta')
take_logs = ['gas', 'price', 'income', 'age', 'distance']
# take logs of continuous variables; part of pre-processing
for label in take_logs:
	gas_df[label] = np.log(gas_df[label])

gas_df['price2']=gas_df['price']**2
spec=1
# specification
if spec == 1:
    factor_list = 'C(driver) + C(hhsize) + C(month) + C(prov) + C(year)'
    rhs_termlist = 'price + I(price**2) + (price:(' + factor_list + ')) + (I(price**2):(' \
        + factor_list + ')) + ' + factor_list + ' + urban + youngsingle + distance + I(distance**2) + age + I(age**2) + income + I(income**2)'
elif spec == 0:
    factor_list = 'C(driver) + C(hhsize) + C(month) + C(prov) + C(year) + age + age**2 + income + income**2'
    rhs_termlist = 'price + price**2 + (price : (' + factor_list + ')) + (price**2 : (' \
                + factor_list + ')) + ' + factor_list + ' + urban + youngsingle + distance + distance**2'

quintile=0
output_var = 'gas'
x_vars = ['price']

Y, X, X_up, X_down, delta = gen_dataset.gen_dataset(rhs_termlist, gas_df, output_var, x_vars, quintile )
p = X.shape[1]
n = X.shape[0]

(5001, 92)


In [13]:
rhs_termlist

'price + (price**2) + (price:(C(driver) + C(hhsize) + C(month) + C(prov) + C(year))) + ((price**2):(C(driver) + C(hhsize) + C(month) + C(prov) + C(year))) + C(driver) + C(hhsize) + C(month) + C(prov) + C(year) + urban + youngsingle + distance + distance**2 + age + age**2 + income + income**2'

In [7]:
import config
from scipy.stats import norm
config.c * norm.ppf(1 - config.alpha / (2 * p)) / np.sqrt(n)

0.023076947625075304

In [64]:
index_list = list(range(0, 29)) +  [56] + list(range(84, 91)) +  list(range(29, 56)) +  list(range(57, 84))
X_r_order = X[:, index_list]
X_r_order.shape

(5001, 91)

In [62]:
list(range(0, 29)) + [56]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 56]

In [91]:
import primitives
stage_1 = reload(stage_1)

def RMD_lasso(M, G, D, _lambda=0, control = {"max_iter":10, "optTol":1e-5, "zeroThreshold":1e-6},
     beta_start = None):
    p = G.shape[1] # num columns 
    Gt = G
    Mt = M
    L = np.concatenate([np.array([config.l]), np.ones(p-1)])
    lambda_vec = _lambda*L*D
    if beta_start is None: # Warm start; allows passing in previous beta
        beta = np.zeros(p)
    else:
        beta = beta_start
    wp = beta
    mm = 1
    k_0 = 0
    k_2 = 0
    k_3 = 0
    while mm < control['max_iter']:
        beta_old = beta.copy()
        for j in range(p):
            rho = Mt[j] - Gt[j, :] @ beta + Gt[j,j]*beta[j]
            z = Gt[j,j]
            if np.isnan(rho):
                beta[j] = 0
                continue
            if rho < -1 * lambda_vec[j]:
                beta[j] = (rho+lambda_vec[j])/z
            if (np.abs(rho) <= lambda_vec[j]):
                beta[j] = 0
            if (rho > lambda_vec[j]):
                beta[j] = (rho-lambda_vec[j])/z
        wp = np.c_[wp, beta]
        if (np.nansum(np.abs(beta - beta_old)) < control['optTol']):
            break
        mm = mm + 1
        print(beta)
    w = beta
    w[np.abs(w) < control['zeroThreshold']] = 0
    return w, wp, mm # returns coefficients, list of past coefficients, and number of steps 


def RMD_stable(Y,X,X_up,X_down,delta,p0,D_LB,D_add,max_iter,is_alpha,is_lasso):
    n = X.shape[0] # num rows
    p = X.shape[1] # num columns

    # First, find low-dimensional moments
    X0 = X[:, 0:p0]
    X0_up = X_up[:, 0:p0]
    X0_down = X_down[:, 0:p0]
    M_hat0, N_hat0, G_hat0, B0 = primitives.get_MNG(Y, X0, X0_up, X0_down, delta)
#     print(M_hat0, N_hat0, G_hat0, B0)
    # initial estimate
    rho_hat0 = np.linalg.solve(G_hat0, M_hat0)
    rho_hat = np.concatenate([rho_hat0, np.zeros(p - G_hat0.shape[1])]) 
    beta_hat0 = np.linalg.solve(G_hat0, N_hat0)
    beta_hat = np.concatenate([beta_hat0, np.zeros(p - G_hat0.shape[1])]) 
    # Full moments
    M_hat, N_hat, G_hat, B = primitives.get_MNG(Y, X, X_up, X_down, delta)
    # penalty term 
    _lambda = config.c * norm.ppf(1 - config.alpha / (2 * p)) / np.sqrt(n)
    if is_alpha:
        ###########
        # alpha_hat
        ###########
        diff_rho=1
        k=1
        rho_tracker = []
        while (diff_rho>config.tol) & (k<=max_iter):
            # previous values
            rho_hat_old=rho_hat.copy()
            
            # normalization
            D_hat_rho=stage_1.get_D(Y,X,X_up,X_down,delta,primitives.m_diff,rho_hat_old)
            D_hat_rho=np.maximum(D_LB,D_hat_rho) 
            D_hat_rho=D_hat_rho+D_add
            if is_lasso:
                rho_hat = RMD_lasso(M_hat, G_hat, D_hat_rho, _lambda)[0]
#                 print(rho_hat)
            else:
                return "dantzig selector not implemented"
                beta_hat = stage_1.RMD_dantzig(M_hat, G_hat, D_hat_rho, _lambda)[0]
            # difference
#             assert(False)
            diff_rho=primitives.two_norm(rho_hat-rho_hat_old)
            rho_tracker.append(diff_rho)
            k=k+1
        print('k: ' + str(k))
        return rho_hat, rho_tracker
    else:
        ###########
        # gamma_hat
        ###########
        diff_beta=1
        k=1
        beta_tracker = []
        while (diff_beta>config.tol) & (k<=max_iter):
            # previous values
            beta_hat_old=beta_hat.copy()
            
            # normalization
            D_hat_beta=stage_1.get_D(Y,X,X_up,X_down,delta,primitives.m2,beta_hat_old)
            D_hat_beta=np.maximum(D_LB,D_hat_beta) # What is this? 
            D_hat_beta=D_hat_beta+D_add
            if is_lasso:
                beta_hat = RMD_lasso(N_hat, G_hat, D_hat_beta, _lambda)[0]
            else:
                return "dantzig selector not implemented"
                beta_hat = stage_1.RMD_dantzig(N_hat, G_hat, D_hat_beta, _lambda)[0]
            # difference
            return()
            diff_beta=primitives.two_norm(beta_hat-beta_hat_old)
            beta_tracker.append(diff_beta)
            k=k+1
        print('k: ' + str(k))
        return beta_hat, beta_tracker
RMD_stable(Y, X_r, X_up_r, X_down_r, delta, p0, D_LB, D_add, max_iter,
             is_alpha=0, is_lasso=(alpha_estimator == 'lasso'))

[ 5.37986763e+00  6.50455676e-02  4.04509265e-01  3.61571002e-01
  0.00000000e+00  0.00000000e+00 -1.06751814e-01  0.00000000e+00
  5.35413112e-02  6.16485910e-02  0.00000000e+00 -5.02884860e-02
  0.00000000e+00 -4.54046369e-02  0.00000000e+00 -2.47653424e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.43217654e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -7.47107591e-02 -2.47923502e-02
  9.27004630e-03 -1.03455996e-02 -4.51094461e-04  0.00000000e+00
  0.00000000e+00 -1.67524163e-01 -8.65957737e-03  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

()

In [98]:
import stage_2
# Y, X, X_up, X_down, delta, p0, D_LB, D_add, max_iter, alpha_estimator, gamma_estimator, bias
stage_2.rrr(Y,X_r,X_up_r,X_down_r,delta,
            p0,D_LB,D_add,max_iter,alpha_estimator,"lasso", 0)

k: 11
k: 11
k: 11
k: 11
k: 11
k: 11
k: 11
k: 11
k: 11
k: 11
[-0.3234838  -0.76389521  1.14925595 ... -1.52942857 -3.73458669
  2.49309699]


(5001, -0.14289267508121448, 0.05053818035404757)

In [80]:
RMD_stable(Y, X_r, X_up_r, X_down_r, delta, p0, D_LB, D_add, max_iter,
             is_alpha=1, is_lasso=(alpha_estimator == 'lasso'))

10000
758
1062
1212
1216
1216
1216
1216
1216
1216
k: 11


(array([  39.16255063,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.27881956,    0.        ,
           0.        ,    0.        ,    0.        ,    2.06488351,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    9.14670644,
           4.81287044,    5.69455933,    0.        ,    0.        ,
           0.        , -122.37826831,    0.71455345,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    8.49330954,    9.95013225,   11.46739748,
           0.        ,    4.19328824,    0.        ,    0.        ,
           0.        ,    0.        ,    0.     

In [4]:
p = X.shape[1]
n = X.shape[0]
#p0=dim(X0) used in low-dim dictionary in the stage 1 tuning procedure
p0=np.ceil(p/4) #p/2 works for low p
if p > 60:
  p0 = np.ceil(p/40)
p0 = int(p0)
D_LB=0 #each diagonal entry of \hat{D} lower bounded by D_LB
D_add=.2 #each diagonal entry of \hat{D} increased by D_add. 0.1 for 0, 0,.2 otw
max_iter=10 #max number iterations in Dantzig selector iteration over estimation and weights

alpha_estimator="lasso"
gamma_estimator="lasso"
bias=0
#alpha_estimator: 0 dantzig, 1 lasso
#gamma_estimator: 0 dantzig, 1 lasso, 2 rf, 3 nn

# set.seed(1) # for sample splitting

import stage_2
stage_2 = reload(stage_2)
results = stage_2.rrr(Y, X, X_up, X_down, delta, p0,
    D_LB,D_add,max_iter,alpha_estimator,gamma_estimator,bias)
print(results)

k: 3
k: 2
k: 3
k: 2
k: 3
k: 2
k: 3
k: 2
k: 3
k: 2
(5001, 0.028784126768979075, 5.794978198355045e-05)


In [19]:
np.sort(A)

array([-3.07792172e+00, -1.82762546e+00, -1.32712868e+00, -1.14037950e+00,
       -1.05882855e+00, -9.99702252e-01, -6.72761673e-01, -6.15623546e-01,
       -5.43351980e-01, -5.36793168e-01, -3.62240619e-01, -3.40738156e-01,
       -3.12677883e-01, -3.10163378e-01, -2.70866290e-01, -2.49107550e-01,
       -2.44018279e-01, -2.41346322e-01, -2.37843444e-01, -2.35287323e-01,
       -2.30884034e-01, -2.24415300e-01, -2.23706019e-01, -2.16354200e-01,
       -1.08356087e-01, -8.36734140e-02, -8.77571400e-03, -2.11992000e-03,
        1.34026800e-03,  3.36554800e-03,  5.20297200e-03,  1.52119070e-02,
        4.97485830e-02,  6.45620660e-02,  1.17223116e-01,  1.20520729e-01,
        1.28995841e-01,  1.31521272e-01,  1.35580807e-01,  1.36186208e-01,
        1.37379338e-01,  1.43252268e-01,  1.44384859e-01,  1.50003836e-01,
        1.51592037e-01,  1.58041858e-01,  1.86358447e-01,  1.88931961e-01,
        1.90477445e-01,  2.09200698e-01,  2.09326932e-01,  2.90467024e-01,
        3.16332376e-01,  

In [20]:
np.sort(N_hat)

array([-3.07792174e+00, -1.82762548e+00, -1.32712869e+00, -1.14037951e+00,
       -1.05882855e+00, -9.99702260e-01, -6.72761678e-01, -6.15623550e-01,
       -5.43351985e-01, -5.36793173e-01, -3.62240622e-01, -3.40738159e-01,
       -3.12677886e-01, -3.10163381e-01, -2.70866292e-01, -2.49107552e-01,
       -2.44018281e-01, -2.41346324e-01, -2.37843446e-01, -2.35287326e-01,
       -2.30884035e-01, -2.24415301e-01, -2.23706021e-01, -2.16354201e-01,
       -1.08356087e-01, -8.36734145e-02, -8.77571444e-03, -2.11992035e-03,
        1.34026781e-03,  3.36554775e-03,  5.20297205e-03,  1.52119072e-02,
        4.97485831e-02,  6.45620669e-02,  1.17223116e-01,  1.20520730e-01,
        1.28995842e-01,  1.31521273e-01,  1.35580808e-01,  1.36186208e-01,
        1.37379338e-01,  1.43252269e-01,  1.44384860e-01,  1.50003837e-01,
        1.51592039e-01,  1.58041859e-01,  1.86358448e-01,  1.88931965e-01,
        1.90477448e-01,  2.09200700e-01,  2.09326935e-01,  2.90467027e-01,
        3.16332380e-01,  

In [36]:
import config 
print(config.L)

5


In [42]:

import numpy as np
from numpy import linalg as LA
from sklearn.model_selection import KFold

base = "/Users/max/Documents/ag_production_functions/code/riesz_representer/rr_python/" 

import os
import sys
sys.path.append(os.path.abspath(base))
import config
import stage_1
import primitives

def rrr(Y, X, X_up, X_down, delta, p0, D_LB, D_add, max_iter, alpha_estimator, gamma_estimator, bias):
    n=X.shape[0]
    kf = KFold(n_splits=config.L, shuffle=True, random_state=config.random)
    Psi_tilde = None
    for l_index, nl_index in kf.split(X):
        Y_l = Y[l_index]
        Y_nl = Y[nl_index]

        X_l = X[l_index, :]
        X_nl = X[nl_index, :]

        X_up_l = X_up[l_index, :]
        X_up_nl = X_up[nl_index, :]
        
        X_down_l = X_down[l_index, :]
        X_down_nl = X_down[nl_index, :]

        n_l = X_l.shape[0]
        n_nl = X_nl.shape[0]

        alpha_hat, gamma_hat = stage_1.get_stage_1(Y_nl,X_nl,X_up_nl,X_down_nl,delta,
            p0,D_LB,D_add,max_iter,alpha_estimator,gamma_estimator)
        if gamma_estimator == "nnet": #nn; here, the derivative is learned during training 
            def m(y, x, x_up, x_down, delta, gamma):
                return gamma_hat.get_derivative(x)
        else: # Otherwise, we use the partial difference to approximate m()
            def m(y, x, x_up, x_down, delta, gamma):
                return m_diff(y, x, x_up, x_down, delta, gamma)
          #   Psi_tilde_l=rep(0,n_l)
        gamma_predictor = gamma_hat.predict if gamma_estimator ==3 else gamma_hat
        if bias:
            Psi_tilde_l = [primitives.psi_tilde_bias(Y_l[i],X_l[i, :],X_up_l[i, :],X_down_l[i, :],
                    delta,m,alpha_hat,gamma_predictor) for i in range(n_l)]# naive plug-in estimator
        else:
            Psi_tilde_l = [primitives.psi_tilde(Y_l[i],X_l[i, :],X_up_l[i, :],X_down_l[i, :],
                    delta,m,alpha_hat,gamma_predictor) for i in range(n_l)]# DML approach
        if Psi_tilde is None:
            Psi_tilde = Psi_tilde_l
        else:
            Psi_tilde = np.concat([Psi_tilde, Psi_tilde_l])

    avg_derivative=np.mean(Psi_tilde)
  
    #influences
    Psi=Psi_tilde-avg_derivative

    var=np.mean(Psi**2)
    se=np.sqrt(var/n)
    return (n, avg_derivative, se)
rrr(Y, X, X_up, X_down, delta, p0,
    D_LB,D_add,max_iter,alpha_estimator,gamma_estimator,bias)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/series.py:842: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


KeyError: 0

In [29]:
np.concatenate([X_down[0:2, :], X_down[2:4, :]]).shape

(2, 94)

In [47]:
N = np.zeros((X.shape[1], X.shape[0]))
N[:, 0] = np.ones(X.shape[1])
N[:, 0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [87]:
def get_MNG(Y, X, X_up, X_down, delta):
    """
    Finds partial difference, using m_diff
    """
    n = X.shape[0]
    p = X.shape[1]

    M=np.zeros((p, n))
    N=np.zeros((p, n))

    for i in range(n):
        N[:,i]=Y[i] * X[i,:] #since m2(w,b)=y*x
        M[:,i] = (X_up[i,:]-X_down[i,:])/delta
    G_hat = X.T @ X
    M_hat = np.mean(M, 1) # Row means 
    N_hat = np.mean(N, 1)
    return M_hat, N_hat, G_hat, X # uncomment to return B as well 
get_MNG(Y, X, X_up, X_down, delta)[0].shape

(47,)

In [55]:
Y.head()

23    5.549076
30    7.037467
34    6.104347
49    4.399375
88    5.662961
Name: gas, dtype: float32

In [16]:
np.unique(X_up)

array([0, 1, 2, 3, 4])

In [29]:
Y[X_up == X_down - 1]

23      5.549076
30      7.037467
34      6.104347
49      4.399375
88      5.662961
105     5.684939
111     5.730100
112     5.912962
120     4.405499
123     5.706446
133     6.148041
145     5.785670
148     5.191845
179     6.481884
190     7.078510
236     5.695078
264     5.426711
277     5.270946
291     5.953243
312     6.624862
319     5.682219
320     5.718999
322     5.463832
329     4.928702
355     5.842384
362     6.567235
373     5.839187
403     6.079933
410     5.517453
416     4.621044
          ...   
4686    5.968196
4721    4.909709
4722    4.010963
4727    5.556056
4750    6.266441
4756    6.209796
4759    4.757032
4760    5.739150
4761    5.744925
4781    4.627910
4782    6.786491
4783    5.650382
4787    6.090405
4814    6.468009
4827    6.076725
4848    7.156644
4854    5.330785
4855    5.105946
4862    5.320079
4876    5.642616
4887    5.231643
4894    6.048317
4897    5.243333
4906    5.906451
4907    5.205654
4916    5.056246
4925    5.972027
4988    6.0075

In [37]:
np.zeros(24)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [39]:
X.shape[1]

47